In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt  
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import matplotlib.patheffects as path_effects
import matplotlib.font_manager as fm
import matplotlib.patches as patch
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Introducción.

- En este notebook, encontraremos lineas de código para analizar los juegos de la Máquina Cemetera del Cruz Azul.
    - Las fuentes de datos es un archivo csv que he extraje de Fivethirtyeight y al cual añadí los disparos totales con ayuda de Fbref.com
    - Para utilizarlo pueden descargar el archivo a su equipo o usar la liga que proporcionar en las primeras líneas de comandos

In [2]:
# liga donde obtendremos los datos del archivo

link = 'https://raw.githubusercontent.com/EduardoSanchezZ/Proyecto-Sports-Analytics/main/Data/Cruz%20Azul/Juegos_de_Cruz_Azul.csv'
data = pd.read_csv(link)
data[data['DT']=='Raul Gutierrez'].head(1)

,ID_Partido,Ciclo,Torneo,ID_Torneo,Fecha,Fase,Condicion,DT,Rival,Pts_pronosticados,Goles_anotados,Goles_recibidos,Disparos_realizados,Disparos_permitidos,xGF,xGC,xJG,xJE,xJP
215,216,2022-23,A-22,12,2022-08-27,REG,Local,Raul Gutierrez,Querétaro,1.84,2,1,8,11,1.04,1.9,0.19,0.23,0.59


# Variables del dataset

    - ID_Partido: Número de partido de la base de datos
    - Ciclo: Año futbolístico (temporada)
    - Torneo: Nombre del torneo
    - Fecha: Fecha del juego
    - Fase: Regular o Liguilla
    - Condición: Local o visitante
    - DT: Director Técnico
    - Rival: Equipo al que se enfrentan 
    - Pts_pronosticados: Proyección de Fivethirtyeight
    - Goles_anotados: Goles anotados en el partido
    - Goles_recibidos: Goles recibidos en el partido
    - Disparos_realizados: Disparos realizados por jugadores de Cruz Azul
    - Disparos_permitidos: Disparos realizados por jugadores rivales
    - xGF: Probabilidad acumulada de que los disparos de Cruz Azul terminen en gol 
    - xGC: Probabilidad acumulada de que los disparos rivales terminen en gol
    - xJG, xJE, xJP: Haciendo una simulación, calculamos la probabilidad de que Cruz Azul gané, empate o pierda

# Agrupación por DT
    - Uno de los análisis más frecuentes es el analizar el desempeño del equipo desglosado por Director Técnico

In [4]:
# Creamos un dataframe nuevo

df = data.copy()

# Agregamos variables para victorias, derrotas, empates y puntos

df['jg'] = np.where(df['Goles_anotados']>df['Goles_recibidos'],1,0)
df['je'] = np.where(df['Goles_anotados']==df['Goles_recibidos'],1,0)
df['jp'] = np.where(df['Goles_anotados']<df['Goles_recibidos'],1,0)
df['pts'] = np.where(df['Goles_anotados']>df['Goles_recibidos'],3,np.where(df['Goles_anotados']<df['Goles_recibidos'],0,1))
df['xpts'] = df['xJG'] * 3 + df['xJE']

# Agrupamos por Director Técnico (DT), debido a que hay diferentes muestras de juegos, utilizaremos medias

dt_agg = df.groupby(['DT']).agg(juegos = ('Fecha', 'count'),
                              exp_jg = ('xJG', 'sum'),
                              exp_je = ('xJE', 'sum'),
                              exp_jp = ('xJP', 'sum'),
                              exp_pts_prom = ('xpts', 'mean'),
                              ganados = ('jg', 'sum'),
                              empates = ('je', 'sum'),
                              derrotas = ('jp', 'sum'),
                              pts_prom = ('pts', 'mean')).reset_index().sort_values(by=['juegos'], ascending=[False])

dt_agg['exp_jg'] = round(dt_agg['exp_jg'],1)
dt_agg['exp_je'] = round(dt_agg['exp_je'],1)
dt_agg['exp_jp'] = round(dt_agg['exp_jp'],1)
dt_agg['exp_pts_prom'] = round(dt_agg['exp_pts_prom'],3)
dt_agg['pts_prom'] = round(dt_agg['pts_prom'],3)

dt_agg = dt_agg.reset_index(drop=True)

dt_agg.index = dt_agg.index + 1

#Output

# Mensaje 
print("Desempeño de Cruz Azul, desglosado por Director Téçnico:")

dt_agg

Desempeño de Cruz Azul, desglosado por Director Téçnico:


,DT,juegos,exp_jg,exp_je,exp_jp,exp_pts_prom,ganados,empates,derrotas,pts_prom
1,Pedro Caixinha,67,28.2,17.6,21.2,1.526,29,22,16,1.627
2,Juan Reynoso,61,23.9,17.0,20.0,1.456,29,17,15,1.705
3,Robert Siboldi,41,18.4,9.4,13.1,1.576,21,7,13,1.707
4,Francisco Jemez,36,16.0,10.5,9.5,1.623,12,14,10,1.389
5,Diego Aguirre,10,2.6,2.4,5.0,1.025,2,2,6,0.800
6,Raul Gutierrez,9,3.7,2.2,3.2,1.473,6,2,1,2.222


# Desempeño en las áreas

- Utilizando la información de disparos intentados, goles esperados y goles anotados mediremos la eficiencia en las áreas de la Máquina

In [27]:
# Desempeño con los disparos:

xg_agg = df.groupby(['DT']).agg(juegos = ('Fecha', 'count'),
                              disp_favor = ('Disparos_realizados', 'mean'),
                              xG_favor = ('xGF', 'mean'), 
                              Goles_favor = ('Goles_anotados', 'mean'),  
                              disp_contra = ('Disparos_permitidos', 'mean'),
                              xG_contra = ('xGC', 'mean'),
                              Goles_contra = ('Goles_recibidos', 'mean')).reset_index().sort_values(by='juegos', ascending=[False])


# Reajustamos el indice

xg_agg = xg_agg.reset_index(drop=True)
xg_agg.index = xg_agg.index + 1

# Output:

print("Disparos por partido: probabilidades y realidades:")

round(xg_agg[xg_agg['DT'].isin(['Diego Aguirre', 'Raul Gutierrez'])].set_index('DT').transpose(), 2)

Disparos por partido: probabilidades y realidades:


DT,Diego Aguirre,Raul Gutierrez
juegos,10.00,9.00
disp_favor,11.80,11.89
xG_favor,1.16,1.44
Goles_favor,1.20,1.67
disp_contra,14.60,11.00
xG_contra,1.73,1.16
Goles_contra,2.50,1.00
